# Степенной метод

In [59]:
import numpy as np

In [60]:
def power_iteration(A: np.matrix, num_iterations: int) -> np.matrix:
    # Ideally choose a random vector
    # To decrease the chance that our vector
    # Is orthogonal to the eigenvector
    b_k = np.random.rand(A.shape[1], 1)
    # b_k = np.matrix("1;"
    #                 "1")

    # print(A, "\n", b_k)

    for _ in range(num_iterations):
        # calculate the matrix-by-vector product Ab
        b_k1 = np.dot(A, b_k)

        # calculate the norm
        b_k1_norm = np.linalg.norm(b_k1)

        # re normalize the vector
        b_k = b_k1 / b_k1_norm

    return b_k

In [61]:
def calc(A: np.matrix, b: np.matrix) -> np.matrix:
    return float((b.T @ A @ b) 
                 / 
                 (b.T @ b))

In [62]:
A: np.matrix = np.matrix("2 1 0;"
              "0 2 1;"
              "0 0 2")
r_k1 = power_iteration(A, 100000)

In [63]:
mu_k1: float = float((r_k1.T @ A @ r_k1) / (r_k1.T @ r_k1))
mu_k1

C:\Users\artem\AppData\Local\Temp\ipykernel_11496\1627451438.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mu_k1: float = float((r_k1.T @ A @ r_k1) / (r_k1.T @ r_k1))


2.000040000287254

In [64]:
A: np.matrix = np.matrix("1 2;"
                         "1 0")
r_k1 = power_iteration(A, 100000)
mu_k1: float = calc(A, r_k1)
mu_k1

C:\Users\artem\AppData\Local\Temp\ipykernel_11496\2676492484.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float((b.T @ A @ b)


2.0

# Обратный степенной

In [65]:
def inverse_power_iteration(A: np.matrix, num_iterations: int, b_k: np.matrix = None) -> np.matrix:
    if b_k is None:
        b_k = np.random.rand(A.shape[1], 1) # Попробовать близкий к настоящему собственный вектор
    print(b_k, "\n")
    
    

    # print(A, "\n", b_k)
    E = np.eye(A.shape[0], dtype=int)
    mu = 0 # попробовать float
    for iter_index in range(num_iterations):

        # print(A - mu * E)
        inv = np.invert(A - mu * E)

        b_top = inv @ b_k
        # print(b_top)
        c_k = np.linalg.norm(b_top)
        # print(b_top, c_k, b_top / c_k)
        # print(b_top / c_k)

        b_k = b_top / c_k
        # c_k не становится меньше 1
        if c_k < 1:
            break
        
        if iter_index % 100 == 0:
            print(b_k)

    print("\n", b_k)
    return b_k

In [66]:
A: np.matrix = np.matrix("1 3 -2 0;"
                         "1 1 4 7;"
                         "4 7 11 23;"
                         "52 66 2 0")  # -0.65
A, calc(A, power_iteration(A, 1000)), calc(A, inverse_power_iteration(A, 10))

[[5.91343393e-02]
 [7.67972330e-04]
 [8.89456951e-01]
 [1.75284713e-01]] 

[[ 0.03406665]
 [-0.34302542]
 [-0.87243038]
 [-0.34646538]]

 [[0.02802362]
 [0.26211264]
 [0.73754927]
 [0.62171755]]


C:\Users\artem\AppData\Local\Temp\ipykernel_11496\2676492484.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float((b.T @ A @ b)


(matrix([[ 1,  3, -2,  0],
         [ 1,  1,  4,  7],
         [ 4,  7, 11, 23],
         [52, 66,  2,  0]]),
 29.37066149896965,
 32.51625621358406)

In [67]:
A: np.matrix = np.matrix("1 -2 -1;"
                         "-1 1 1;"
                         "1 0 -1")  # 2 - 0


A,
calc(A, power_iteration(A, 1000)),
calc(A, inverse_power_iteration(A, 10000, np.matrix("0.5;"
                                                    "0;"
                                                    "1")))  # При увеличении итераций всё меняется # 10000 - самое оптимальное

C:\Users\artem\AppData\Local\Temp\ipykernel_11496\2676492484.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float((b.T @ A @ b)


[[0.5]
 [0. ]
 [1. ]] 

[[-0.40824829]
 [-0.81649658]
 [-0.40824829]]
[[-0.3059045 ]
 [-0.74147957]
 [-0.59718547]]
[[-0.24087106]
 [-0.78106215]
 [-0.57612763]]
[[-0.17644144]
 [-0.81475833]
 [-0.5523018 ]]
[[-0.11317767]
 [-0.84279753]
 [-0.52619686]]
[[-0.05151241]
 [-0.86549424]
 [-0.49826318]]
[[ 0.00824213]
 [-0.883213  ]
 [-0.46889964]]
[[ 0.06587934]
 [-0.89633904]
 [-0.43844753]]
[[ 0.12128064]
 [-0.9052555 ]
 [-0.40718973]]
[[ 0.17439756]
 [-0.91032697]
 [-0.37535356]]
[[ 0.2252349 ]
 [-0.91188857]
 [-0.34311584]]
[[ 0.273836  ]
 [-0.91023946]
 [-0.31060902]]
[[ 0.32027018]
 [-0.90563969]
 [-0.27792763]]
[[ 0.36462252]
 [-0.89830926]
 [-0.24513445]]
[[ 0.40698563]
 [-0.88842881]
 [-0.21226623]]
[[ 0.44745316]
 [-0.87614114]
 [-0.17933872]]
[[ 0.48611465]
 [-0.86155319]
 [-0.14635108]]
[[ 0.52305143]
 [-0.84473821]
 [-0.11328973]]
[[ 0.55833322]
 [-0.82573781]
 [-0.08013165]]
[[ 0.59201525]
 [-0.80456403]
 [-0.04684738]]
[[ 0.62413554]
 [-0.7812011 ]
 [-0.0134037 ]]
[[ 0.65471

0.08275701546367463

## Модификация Рэлэя

In [190]:
def rayleigh(A, epsilon, mu, x):
  x = x / np.linalg.norm(x)
  y = np.linalg.solve(A - mu * np.eye(A.shape[0]), x)
  # print(y)
  # print(x)
  lambda_ = float(np.dot(y.T, x))
  # print(lambda_, lambda_ * x)
  mu = mu + 1 / lambda_
  err = np.linalg.norm(y - lambda_ * x) / np.linalg.norm(y)

  while err > epsilon:
    x = y / np.linalg.norm(y)
    y = np.linalg.solve(A - mu * np.eye(A.shape[0]), x)
    lambda_ = float(np.dot(y.T, x))
    mu = mu + 1 / lambda_
    err = np.linalg.norm(y - lambda_ * x) / np.linalg.norm(y)

  return float(mu), x


In [193]:
A: np.matrix = np.matrix("1 -2 -1;"
                         "-1 1 1;"
                         "1 0 -1")  # 2 - 0

pmp = calc(A, power_iteration(A, 1000))

mu_r, egg = rayleigh(A, 0.001, 3, np.matrix("0.5;"
                                             "0;"
                                             "1"))

A,
calc(A, power_iteration(A, 1000)),
# calc(A, inverse_power_iteration(A, 10000, np.matrix("0.5;"
#                                                     "0;"
#                                                     "1")))  # При увеличении итераций всё меняется # 10000 - самое оптимальное

mu_r, egg

C:\Users\artem\AppData\Local\Temp\ipykernel_11496\2676492484.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float((b.T @ A @ b)
C:\Users\artem\AppData\Local\Temp\ipykernel_11496\1041875321.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  lambda_ = float(np.dot(y.T, x))
C:\Users\artem\AppData\Local\Temp\ipykernel_11496\1041875321.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  lambda_ = float(np.dot(y.T, x))


(1.0436626212468496e-12,
 matrix([[7.07107506e-01],
         [6.68093168e-07],
         [7.07106056e-01]]))